In [ ]:
! pip install numpy
! pip install openai==1.2.3
! pip install pymongo
! pip install python-dotenv
! pip install tenacity

## Set up

In [35]:
import json
import openai
import pymongo

from dotenv import dotenv_values
from openai import AzureOpenAI

env_name = "myconfig.env" 
config = dotenv_values(env_name)


# Connection string
cosmos_conn = config['cosmos_connection_string']
cosmos_client = pymongo.MongoClient(cosmos_conn)

# Database name
DATABASE_NAME = "ProductRecommendation"
db = cosmos_client[DATABASE_NAME]

# Collection names
actual_ratings = db["ActualRating"]
predicted_ratings = db["PredictedRating"]
product_catalog = db['ProductCollection']


openai.api_type = config['openai_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

In [106]:
def generate_embeddings(text):
    try:
        response = client.embeddings.create(
            input=text, model="embeddings") # need to read from config
        
        embeddings = response.data[0].embedding
        
        return embeddings
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Product Recommendation Functions

In [112]:
def predictions_from_current_product_page(user_id, current_product_id, num_results=3):
    """
    This function recommends similar products predicted for this user excluding the current product.
    """
    
    # Get the predicted products for the user, limit results
    user_predicted_products = predicted_ratings.find_one( 
        { "UserId": user_id },
        {"Predictions": {"$slice": num_results}})

    # Filter out the current product if exists and return a list of product ids
    product_ids = [prediction['ProductId'] for prediction in user_predicted_products['Predictions'] 
        if prediction['ProductId'] != current_product_id]
    
    predicted_products = []

    # Look up recommended products maintaining order of product_ids as these are in descending order of predicted ratings
    for product_id in product_ids:
        product = product_catalog.find_one({"Id": product_id})
        if product:
            predicted_products.append(product)

    predicted_products = list(predicted_products)

    return predicted_products

In [38]:
def predictions_from_vector_search(user_id, user_query, num_results=10):
    """ 
    This function takes a user prompt search for products and returns products that are predicted for the user. 
    """
    
    # Generate the embedding for the user query
    query_embedding = generate_embeddings(user_query)

    # Get the predicted products for the user
    predicted_products = predicted_ratings.find_one( { "UserId": user_id } )

    # Extract the ProductId from the Predictions array
    product_ids = [prediction['ProductId'] for prediction in predicted_products['Predictions']]

    # Filter criteria to include predicted products
    filter_criteria = { 
        "Id": {"$in": product_ids}
    }

    results = product_catalog.aggregate([
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "Embedding",
                    "k": num_results,
                    "filter": filter_criteria
                },
                "returnStoredSource": True
            }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' }
    }])

    predicted_products = list(results)

    return predicted_products

In [115]:
# actual_user_ids = [144, 496, 189, 232, 194, 950, 370, 980, 190, 404, 737, 959, 142, 795, 121, 743, 307, 365, 30, 726, 339, 536]
# snowboards product_ids = 73, 5, 92, 83, 43, 12, 53, 62, 22
user_query = "I want snowboard like an olympic champion."
user_id = '189'
user_id = int(user_id)
product_id = int(92) # Shaun White snowboard
num_results = 10

print("---------Vector Search Results: --------")
vector_search_with_predictions = predictions_from_vector_search(user_id, user_query, num_results)

for product in vector_search_with_predictions:
    print(f"{product['document']['Id']}: {product['document']['Name']} - {product['document']['Price']}")


print("\n--------Current Page Results: ---------")
on_page_predictions = predictions_from_current_product_page(user_id, product_id, num_results)

for product in on_page_predictions:
    print(f"{product['Id']}: {product['Name']} - {product['Price']}")



---------Vector Search Results: --------
92: Shaun White Powder Groomer - 449.99
73: Omni-Snow Dual Snowboard - 289.99
43: Glacier Frost Snowboard - 419.99
32: Cosmic Purple Snowboard - 419.99
12: Powder Pro Snowboard - 399.0
62: Shadow Black Snowboard - 379.0
22: Venture 2022 Snowboard - 499.0
72: GravityZone All-Mountain Skis - 699.0
82: Maverick Pro Ski Goggles - 139.99
42: Gravity 5000 All-Mountain Skis - 699.0

--------Current Page Results: ---------
42: Gravity 5000 All-Mountain Skis - 699.0
72: GravityZone All-Mountain Skis - 699.0
22: Venture 2022 Snowboard - 499.0
62: Shadow Black Snowboard - 379.0
32: Cosmic Purple Snowboard - 419.99
73: Omni-Snow Dual Snowboard - 289.99
12: Powder Pro Snowboard - 399.0
43: Glacier Frost Snowboard - 419.99
69: Expedition 200 GPS Navigator - 299.0
